# 10. 机器学习

In [32]:
import random

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from matplotlib.animation import FuncAnimation
from sklearn.datasets import fetch_openml

dataset = fetch_openml(name='boston', version=1, as_frame=True, return_X_y=False, parser='pandas')

In [33]:
data = dataset['data']
target = dataset['target']
columns = dataset['feature_names']

dataframe = pd.DataFrame(data)
dataframe.columns = columns
dataframe['price'] = dataset['target']

rm = dataframe['RM']
lstat = dataframe['LSTAT']

In [34]:
greater_then_most = np.percentile(dataframe['price'], 66)
dataframe['expensive'] = dataframe['price'].apply(lambda p: int(p > greater_then_most))
expensive = dataframe['expensive']

In [35]:
def logistic(x):
    return 1 / (1 + np.exp(-x))

def model(x, w, b):
    return logistic(np.dot(x, w.T) + b)

def loss(yhat, y):
    return -np.sum(y * np.log(yhat) + (1 - y) * np.log(1 - yhat))

def partial_w(x, y, yhat):
    return np.array([np.sum((yhat - y) * x[0]), np.sum((yhat - y) * x[1])])

def partial_b(x, y, yhat):
    return np.sum((yhat - y))

In [36]:
import pickle

In [37]:
def train(model_to_be_train, target, loss, pw, pb):
    w = np.random.random_sample((1, 2))
    b = 0

    learning_rate = 1e-5
    epoch = 200
    losses = []

    history_k_b_loss = []

    for i in range(epoch):
        batch_loss = []
        for batch in range(len(rm)):
            index = random.choice(range(len(rm)))

            x = np.array([rm[index], lstat[index]])
            y = expensive[index]

            yhat = model_to_be_train(x, w, b)
            loss_v = loss(yhat, y)

            w = w + -1 * partial_w(x, y, yhat) * learning_rate
            b = b + -1 * partial_b(x, y, yhat) * learning_rate

            batch_loss.append(loss_v)
            history_k_b_loss.append((w, b, loss_v))

        #     if batch % 100 == 0:
        #         print('Epoch: {}, Batch: {}, loss:{}'.format(i, batch, loss_v))
        losses.append(np.mean(batch_loss))
    return model_to_be_train, w, b, losses

In [69]:
model, w, b, losses = train(model, target, loss, partial_w, partial_b)

with open('logistic_regression.model', 'wb') as f:
    pickle.dump(model, f)

with open('w.model', 'wb') as f:
    pickle.dump(w, f)

with open('b.model', 'wb') as f:
    pickle.dump(b, f)

print('pickle write finished')

pickle write finished


In [39]:
import pickle

with open('logistic_regression.model', 'rb') as f:
    model_r = pickle.load(f)

with open('w.model', 'rb') as f:
    w_r = pickle.load(f)

with open('b.model', 'rb') as f:
    b_r = pickle.load(f)

print('pickle read finished')

pickle read finished


In [82]:
true_labels, predicated_labels, loss_labels = [], [], []

In [83]:
random_test_indices = np.random.choice(range(len(rm)), size=100)
decision_boundary = 0.5

for i in random_test_indices:
    x1, x2, y = rm[i], lstat[i], expensive[i]
    predicate = model_r(np.array([x1, x2]), w_r, b_r)
    loss_labels.append(predicate)
    predicate_label = int(predicate > decision_boundary)

    print('RM:{}, LSTAT:{}, EXPENSIVE:{}, Predicated:{}, loss_labels'.format(x1, x2, y, predicate_label), predicate)

    true_labels.append(y)
    predicated_labels.append(predicate_label)

RM:5.707, LSTAT:12.01, EXPENSIVE:0, Predicated:0, loss_labels [0.19626449]
RM:6.657, LSTAT:21.22, EXPENSIVE:0, Predicated:0, loss_labels [0.02528174]
RM:5.895, LSTAT:10.56, EXPENSIVE:0, Predicated:0, loss_labels [0.28031166]
RM:6.121, LSTAT:8.44, EXPENSIVE:0, Predicated:0, loss_labels [0.43124459]
RM:7.333, LSTAT:7.79, EXPENSIVE:1, Predicated:1, loss_labels [0.57760506]
RM:6.081, LSTAT:14.7, EXPENSIVE:0, Predicated:0, loss_labels [0.11586862]
RM:4.903, LSTAT:29.29, EXPENSIVE:0, Predicated:0, loss_labels [0.00151733]
RM:6.083, LSTAT:12.79, EXPENSIVE:0, Predicated:0, loss_labels [0.18242465]
RM:7.61, LSTAT:3.11, EXPENSIVE:1, Predicated:1, loss_labels [0.84665844]
RM:6.8, LSTAT:5.03, EXPENSIVE:1, Predicated:1, loss_labels [0.71118202]
RM:7.47, LSTAT:3.16, EXPENSIVE:1, Predicated:1, loss_labels [0.83855137]
RM:7.249, LSTAT:4.81, EXPENSIVE:1, Predicated:1, loss_labels [0.75283958]
RM:5.841, LSTAT:11.41, EXPENSIVE:0, Predicated:0, loss_labels [0.23189355]
RM:5.933, LSTAT:9.68, EXPENSIVE:0, P

/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/ipykernel_17242/2755839443.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicate_label = int(predicate > decision_boundary)


In [43]:
def accuracy(ytrues, yhats):
    return sum(1 for yt, y1 in zip(ytrues, yhats) if yt == y1) / len(ytrues)

In [44]:
accuracy(true_labels, predicated_labels)

0.89

In [45]:
def precision(ytrues, yhats):
    # 预测标签是 1 的里面，正确的比例是多少

    positives_pred = [y for y in yhats if y == 1]
    return sum(1 for yt, y in zip(ytrues, yhats) if yt == y and y == 1) / len(positives_pred)

In [46]:
precision(true_labels, predicated_labels)

0.8181818181818182

In [47]:
def recall(ytrues, yhats):
    
    true_positive = [y for y in ytrues if y == 1]     
    return sum(1 for yt, y in zip(ytrues, yhats) if yt == y and yt == 1) / len(true_positive)

In [48]:
recall(true_labels, predicated_labels)

0.72

In [49]:
people = [0] * 90 + [1] * 10

In [50]:
import random
random.shuffle(people)

In [51]:
a = [0] * 100
b = [1] * 100

In [52]:
accuracy(people, a)

0.9

In [53]:
precision(people, a)

ZeroDivisionError: division by zero

In [54]:
recall(people, a)

0.0

In [55]:
accuracy(people, b)

0.1

In [56]:
precision(people, b)

0.1

In [57]:
recall(people, b)

1.0

In [87]:
boston_model = [true_labels, predicated_labels, loss_labels]

In [88]:
with open('boston_labels', 'wb') as f:
    pickle.dump(boston_model, f)

In [84]:
len(loss_labels)

100

In [86]:
print(len(true_labels), len(predicated_labels), len(loss_labels))

100 100 100


![欢迎订阅：坍缩的奇点](../assets/Capture-2023-11-02-164446.png)